In [2]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.12 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 36.6/112.6 GB disk)


In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import requests
import cv2
from ultralytics import YOLO
from glob import glob
import os

In [ ]:
train_images_path = "/content/drive/MyDrive/inSitu/dataset/train/images"
train_masks_path = "/content/drive/MyDrive/inSitu/dataset/train/masks"
val_images_path = "/content/drive/MyDrive/inSitu/dataset/val/images"
val_masks_path = "/content/drive/MyDrive/inSitu/dataset/val/masks"

In [ ]:
train_images = sorted(glob(os.path.join(train_images_path, "*")))
train_masks = sorted(glob(os.path.join(train_masks_path, "*")))
val_images = sorted(glob(os.path.join(val_images_path, "*")))
val_masks = sorted(glob(os.path.join(val_masks_path, "*")))
print(len(train_images), len(train_masks))

2208 2208


# Сегменатация (дообучаю yolo на датасете с масками)

Создаем текстовые представления для датасета для дообучения

In [ ]:
def mask_to_polygons(mask_path):
    mask = cv2.imread(mask_path, 0)

    _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for contour in contours:
        contour = contour.flatten().tolist()
        polygons.append(contour)

    return polygons

In [ ]:
def normalize_polygon(polygon, img_size):
    img_w, img_h = img_size
    normalized_polygon = []
    for i in range(0, len(polygon), 2):
        x = polygon[i] / img_w
        y = polygon[i + 1] / img_h
        normalized_polygon.append(x)
        normalized_polygon.append(y)
    return normalized_polygon

In [ ]:
train_labels_path = "/content/drive/MyDrive/inSitu/dataset/train/labels/"
val_labels_path = "/content/drive/MyDrive/inSitu/dataset/val/labels/"

In [ ]:
def create_yolo_annotations(image_dir, mask_dir, output_dir, class_id=0):
    i = 0
    for img_file in image_dir:
        if img_file.endswith(".png"):
            img_path = img_file
            mask_path = mask_dir[i]
            label_path = output_dir + img_file.replace('.png', '.txt').split('/')[-1]

            img = cv2.imread(img_file)
            img_h, img_w, _ = img.shape

            polygons = mask_to_polygons(mask_path)
            with open(label_path, 'w') as f:
              for polygon in polygons:
                normalized_polygon = normalize_polygon(polygon, (img_w, img_h))
                f.write(f"{class_id} " + " ".join(map(str, normalized_polygon)) + "\n")
        i += 1

In [ ]:
create_yolo_annotations(train_images, train_masks, train_labels_path)

In [ ]:
create_yolo_annotations(val_images, val_masks, val_labels_path)

In [ ]:
train_h = os.path.join('/content/drive/MyDrive/inSitu/dataset/train')
train_h

'/content/drive/MyDrive/inSitu/dataset/train'

In [ ]:
def delete_alot_h(h_dir):
    for h_file in h_dir:
        if h_file.endswith(".txt"):
          os.remove(h_file)

In [ ]:
delete_alot_h(train_h)

Подбор гиперпараметров(нужно запустить только 3 ячейки, которые ниже)

In [4]:
import pandas as pd
import numpy as np
from PIL import Image
import requests
import cv2
from ultralytics import YOLO
from glob import glob
import os

In [ ]:
model_g = YOLO('yolo11n-seg.pt')
model_g.tune(data="/content/drive/MyDrive/inSitu/dataset_con.yaml", epochs=30, iterations=300, optimizer="AdamW", plots=True, save=True, val=True)

100%|██████████| 5.90M/5.90M [00:00<00:00, 113MB/s]


Tuner: Initialized Tuner instance with 'tune_dir=runs/segment/tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/300 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}


Дообучаем модель

In [ ]:
model_30 = YOLO('yolo11n-seg.pt')
model_30.train(data='/content/drive/MyDrive/inSitu/dataset_con.yaml', epochs=30, save_period=1)

Ultralytics 8.3.9 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=/content/drive/MyDrive/inSitu/dataset_con.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning /content/drive/MyDrive/inSitu/dataset/train/labels.cache... 2208 images, 11 backgrounds, 0 corrupt: 100%|██████████| 2208/2208 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/inSitu/dataset/val/labels.cache... 552 images, 13 backgrounds, 0 corrupt: 100%|██████████| 552/552 [00:00<?, ?it/s]


Plotting labels to runs/segment/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/segment/train3
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        552        743      0.893      0.795      0.855      0.622      0.898       0.79      0.859      0.586



30 epochs completed in 0.846 hours.
Optimizer stripped from runs/segment/train3/weights/last.pt, 6.0MB
Optimizer stripped from runs/segment/train3/weights/best.pt, 6.0MB

Validating runs/segment/train3/weights/best.pt...
Ultralytics 8.3.9 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:19<00:00,  1.09s/it]


                   all        552        743      0.901      0.798      0.856      0.622      0.903      0.798      0.864      0.592
Speed: 0.6ms preprocess, 4.7ms inference, 0.0ms loss, 4.8ms postprocess per image
Results saved to runs/segment/train3


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e46ce9f6b00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [ ]:
model_30.save('/content/drive/MyDrive/inSitu/yolo_30_wound.pt')

# Визуализация

In [ ]:
colors = [
    (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255),
    (255, 0, 255), (192, 192, 192), (128, 128, 128), (128, 0, 0), (128, 128, 0),
    (0, 128, 0), (128, 0, 128), (0, 128, 128), (0, 0, 128), (72, 61, 139),
    (47, 79, 79), (47, 79, 47), (0, 206, 209), (148, 0, 211), (255, 20, 147)
]

def process_image(image_path):
    image = cv2.imread(image_path)
    image_orig = image.copy()
    h_or, w_or = image.shape[:2]
    image = cv2.resize(image, (640, 640))
    results = model_30(image)[0]

    classes_names = results.names
    classes = results.boxes.cls.cpu().numpy()
    masks = results.masks.data.cpu().numpy()


    for i, mask in enumerate(masks):
        color = colors[int(classes[i]) % len(colors)]


        mask_resized = cv2.resize(mask, (w_or, h_or))


        color_mask = np.zeros((h_or, w_or, 3), dtype=np.uint8)
        color_mask[mask_resized > 0] = color


        mask_filename = '/content/drive/MyDrive/inSitu/results/' + os.path.splitext(os.path.basename(image_path))[0] + f"_{classes_names[classes[i]]}_{i}.png"
        cv2.imwrite(mask_filename, color_mask)

        image_orig = cv2.addWeighted(image_orig, 1.0, color_mask, 0.5, 0)


    new_image_path = '/content/drive/MyDrive/inSitu/results/' + os.path.splitext(os.path.basename(image_path))[0] + '_segmented' + os.path.splitext(image_path)[1]
    cv2.imwrite(new_image_path, image_orig)
    print(f"Segmented image saved to {new_image_path}")

wound = process_image('/content/drive/MyDrive/inSitu/dataset/280h.png')


0: 640x640 2 wounds, 21.0ms
Speed: 2.1ms preprocess, 21.0ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
Segmented image saved to /content/drive/MyDrive/inSitu/results/280h_segmented.png


In [ ]:
process_image('/content/drive/MyDrive/inSitu/dataset/250h.png')


0: 640x640 1 wound, 17.0ms
Speed: 2.1ms preprocess, 17.0ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)
Segmented image saved to /content/drive/MyDrive/inSitu/results/250h_segmented.png


In [ ]:
process_image('/content/drive/MyDrive/inSitu/dataset/250m.png')


0: 640x640 2 wounds, 25.2ms
Speed: 3.5ms preprocess, 25.2ms inference, 4.8ms postprocess per image at shape (1, 3, 640, 640)
Segmented image saved to /content/drive/MyDrive/inSitu/results/250m_segmented.png


In [ ]:
process_image('/content/drive/MyDrive/inSitu/dataset/350h.png')


0: 640x640 2 wounds, 16.9ms
Speed: 2.3ms preprocess, 16.9ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 640)
Segmented image saved to /content/drive/MyDrive/inSitu/results/350h_segmented.png
